In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: Generate the true class balance to be recovered

In [2]:
K = 2

# Generate the true class balance to be recovered
p_Y = np.random.random(K)
p_Y /= p_Y.sum()
p_Y

array([0.65640301, 0.34359699])

### Step 2: Generate the true conditional probability tables (CPTs) for the LFs

Separate simple process here to keep simple (later merge this with the SPA generator).
Generate in terms of the _conditional accuracies_ (which is equivalent to the recall...):
$$
\alpha_{i,y',y} = P(\lambda_i = y' | Y = y)
$$

Note that this table should be normalized such that:
$$
\sum_{y'} \alpha_{i,y',y} = 1
$$

In [3]:
M = 25
alphas = []
for i in range(M):
    a = np.random.random((K,K))
    alphas.append( a @ np.diag(1 / a.sum(axis=0)) )
alpha = np.array(alphas)

assert np.all(np.abs(alpha.sum(axis=1) - 1) < 1e-5)
alpha[0]

array([[0.60167121, 0.28668221],
       [0.39832879, 0.71331779]])

### Aside: Different tensor product approaches in Python

#### (1) Brute force

In [4]:
%%time
O_1 = np.zeros((M,M,K,K))
for i, j, y1, y2 in product(range(M), range(M), range(K), range(K)):
    for y in range(K):
        O_1[i,j,y1,y2] += alpha[i, y1, y] * alpha[j, y2, y]

CPU times: user 7.63 ms, sys: 258 µs, total: 7.88 ms
Wall time: 7.93 ms


#### (2) `np.tensordot`

In [5]:
%time O_2 = np.swapaxes(np.tensordot(alpha, alpha, axes=(2,2)), 1, 2)

CPU times: user 714 µs, sys: 361 µs, total: 1.07 ms
Wall time: 658 µs


In [6]:
np.mean(np.abs(O_1 - O_2))

1.2656542480726785e-17

#### (3) `np.einsum`

In [7]:
%time O_3 = np.einsum('iay,jby->ijab', alpha, alpha)

CPU times: user 259 µs, sys: 120 µs, total: 379 µs
Wall time: 326 µs


In [8]:
np.mean(np.abs(O_1 - O_3))

0.0

Now, trying a bilinear form:

In [9]:
%%time
Op_1 = np.zeros((M,M,K,K))
for i, j, y1, y2 in product(range(M), range(M), range(K), range(K)):
    for y in range(K):
        Op_1[i,j,y1,y2] += p_Y[y] * alpha[i, y1, y] * alpha[j, y2, y]

CPU times: user 8.75 ms, sys: 284 µs, total: 9.04 ms
Wall time: 9.17 ms


In [10]:
%time Op_3 = np.einsum('aby,y,cdy->acbd', alpha, p_Y, alpha)

CPU times: user 411 µs, sys: 56 µs, total: 467 µs
Wall time: 420 µs


In [11]:
np.mean(np.abs(Op_1 - Op_3))

5.083433674002436e-18

### Step 3: Generate the pairwise overlaps tensor $O$ of conditionally-independent LFs

Now we can directly generate $O$.
By our conditional independence assumption, we have:
$$
P(\lambda_i = y', \lambda_j = y'' | Y = y) = \alpha_{i,y',y} \alpha_{j,y'',y}
$$

Thus we have:
$$
O_{i,j,y',y''} = \sum_y P(Y=y) \alpha_{i,y',y} \alpha_{j,y'',y}
$$

In [12]:
%%time
O = np.einsum('aby,y,cdy->acbd', alpha, p_Y, alpha)
for i in range(M):
    O[i,i,:,:] = 0.0
O = torch.from_numpy(O).float()

CPU times: user 550 µs, sys: 125 µs, total: 675 µs
Wall time: 578 µs


In [13]:
A = torch.from_numpy(alpha).float()
P = torch.from_numpy(p_Y).float()

In [14]:
mask = torch.ones((M,M,K,K)).byte()
for i in range(M):
    mask[i,i,:,:] = 0

torch.norm((O - torch.einsum('aby,y,cdy->acbd', [A,P,A]))[mask])**2

tensor(5.6265e-13)

### Step 4: Try to recover $\alpha$ given $P(Y=y)$

In [15]:
mask = torch.ones((M,M,K,K)).byte()
for i in range(M):
    mask[i,i,:,:] = 0

def get_loss(A, P, O):
    loss_1 = torch.norm((O - torch.einsum('aby,y,cdy->acbd', [A,P,A]))[mask])**2
    loss_2 = torch.norm(torch.sum(A, 1) - 1)**2
    return loss_1 + loss_2

# Train the model
def train_model(A, P, O, n_epochs=10, lr=0.01, print_every=1):
    optimizer = optim.SGD([A], lr=lr)
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass to calculate outputs
        loss = get_loss(A, P, O)

        # Backward pass to calculate gradients
        loss.backward()

        # Perform optimizer step
        optimizer.step()

        # Keep running sum of losses
        epoch_loss += loss.detach()

        # Report progress
        if epoch % print_every == 0:
            msg = f"[E:{epoch}]\tLoss: {epoch_loss:.8f}"
            print(msg)

In [21]:
A = nn.Parameter(torch.from_numpy(np.random.rand(M, K, K)).float()).float()
P = torch.from_numpy(p_Y).float()

train_model(A, P, O, n_epochs=500, lr=0.01, print_every=100)

print(f"Param estimation error: {np.mean(np.abs(A.detach().numpy() - alpha))}")

[E:0]	Loss: 118.10226440
[E:100]	Loss: 0.58557749
[E:200]	Loss: 0.00237393
[E:300]	Loss: 0.00002269
[E:400]	Loss: 0.00000034
Param estimation error: 6.0834185818131605e-06


In [22]:
A[0]

tensor([[0.6017, 0.2867],
        [0.3983, 0.7133]], grad_fn=<SelectBackward>)

In [23]:
alpha[0]

array([[0.60167121, 0.28668221],
       [0.39832879, 0.71331779]])